In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Superconductivity transition measurement
If you would like to measure the resistance as a function of temperature via the old manual method,
just follow the lab notes, using sand as as the heat sink to make sure that the puck warms up slowly.

Alternatively, try this code, which will automate the data collection.


Connections:

2400 Red/Black leads go to the black wires (either way) of the puck
2182 Red/Black leads go to the yellow wires (either way) of the puck
2182 Green/White leads go to the purple/brown leads of the puck (or the narrow/wide contacts on the thermo-couple connector).  These are the thermocouple connections, so the orientation is important.

Finally, make a connection between the two grounds on the 2182 wires:  make contact between the White lead and the Black lead.  

### If you keep seeing OVERFLOW K readings, you've lost the common ground connection just mentioned.

Now arrange the leads so that they do not touch one another.  Place the green/white thermocouple connections up and off the surface of the table so that cold LN2 boiloff will not change the temperature from ambient (we've been using a foam block).  We are relying upon keeping the temperature of these junctions near room temperature, 22 C, or 295 K.  If the room temperature is significantly different from this value, you'll need to change a line of code in the cell block below--just search for 295 to find the line of code.

Fill a styrofoam cup 1/3 with sand.  Place the puck on top, then bury it with more sand.

#### Turn the Keithley 2400 Sourcemeter output ON, press Sweep, then Trig.  This will start the triggered sweep, and you should see Isrc alternate between + 10 mA and -10 mA.

###  Edit the headertext that will be written out with your data file to correspond to your experiment.
Feel free to modify the number <MAX_COUNT> of samples (800-1500 is good)


Now run this program with the play button, after positioning your cursor on the next cell.

The Keithley 2182 nanovoltmeter should start showing alternating readings of absolute temperature (measured via the T-type thermocouple buried in the center of the puck) and the difference voltage (V1-V2)/2, from which you will deduce the resistance.  If it doesn't work, you might need to power both boxes off, then on.  And if this program doesn't run, try going to Kernel/Restart & Clear Output.  Don't forget to turn the Keithley 2400 Sourcemeter output ON, press Sweep, then Trig.

Weird readings of 9E37 on the temperature result when the common ground (little white and little black) wires are not connected.

Fill the styrofoam cup with liquid nitrogen.  Use a pencil to poke through any ice crust that may form over the sand.  Fill the cup again.  Watch that the temperature readings are dropping nicely.  We can get down to about 83-85K.  Be careful with the connections to the puck--they are a bit delicate, so resist the temptation to wiggle around the wire bundle. 

Now allow the cup to warm up.  If you'd like it to warm up faster, just blow room temperature air across the cup-- there's probably no need to heat the air, unless you are drying the puck off at the end of the data run.


If you get a code error, just try running the code again.

If your run is successful, results should be written to a file in your documents directory, named with a time stamp. If you quit early, you'll want to run a later code block to save the data.

### Look farther down in this notebook for code blocks that will allow you to plot the data directly without using Excel.

#### What you'll see

This program assumes you have a second computer monitor to the left of the main display.  Two large graphics windows will open.  One of them plots Voltage vs. Temperature.  The other plots Temperature vs. Data Count.  At the end of the data run, which happens after MAX_COUNT is reached, the displays will remain static until you close them.  You'll know when the run has finished because the marker in the Data Count plot will stop on the right-hand side.  And now it beeps.

Data are written out in the output cell below the code cell.  If all else fails, you should be able to cut and paste these data into another plotting routine or Excel.

If you try out the cells below the main one, you may find that you need to close each figure before plotting a new one.



In [ ]:
# Python routine to read temperature and voltage with the Keithley 2182 nanovoltmeter
# This is for the Superconductivity Lab in Modern Physics
# PJHT 1/1/2015; 11/1/2016
%matplotlib
import matplotlib.pyplot as plt
import serial  #pySerial folder needs to be put in the directory pythonXX\lib\site-packages\serial
#The standard installation lacks this serial package, so open an Anaconda command window, then type
#>>>pip install pyserial
#then check that it's there with 
#>>>pip list
# If you run pip from Python 3 command line, it'll be placed in the proper directory
import winsound
from numpy import append #need this to gather the data on the fly
import datetime
import time
from time import sleep, localtime,strftime #some timekeeping functions

#from time import sleep, localtime,strftime #some timekeeping functions

MAX_COUNT = 600  #change this number if you want a different number of samples.  You might want 800
# don't despair if you stop early--data are saved as data1 and data2 lists, which are non-volatile
headertext = "Time\tTemperature\tVoltage\tfor 10 mA drive current"

#Connections:
# Red / black from Sourcemeter (current source supply, so large cables) goes to black wires of puck
# Red / black from voltmeter (spindly wires for measuring voltage) got yellow wires of puck
# Green/white (channel 2 of voltmeter) go to TC connection block, with green to copper side

#The 2400 sourcemeter and 2182 nanovoltmeter are set up following the procedure on pp. 5-10 to 5-13 in the 2182 manual. 
# Just do Steps 1, 3, 4, 5, 7, 8 to set up a 2-point custom sweep with
# P0000 = +10 mA
# P0001 = -10 mA
# Because this piece of software communicates only with the 2182, you need to manually turn on the OUTPUI of the 2400
# to start everything rolling.  Then do the Sweep/Trigger buttons on the 2400
ERR_MSG_ON = 0  #1 = on , 0 = off

def writekeithley(kstring, ERR_ON):  #write a command string to the keithley
    keithstring = kstring+"\r"  #append <CR>
    # the .encode() method converts string into byte array, and is required to use ser.write() routine
    err = ser.write(keithstring.encode()) #encode as bytes
    sleep(PAUSE)
    if (ERR_ON==True):
        err1 = ser.write(":syst:err?\r".encode())
        data = ser.read(DATA_LENGTH)
        print(keithstring + "  Error return message:",data) 
    return err


PAUSE=0.0 #this is OK at 0.1, at least for 9600 baud
BAUD = 9600  #we can raise this rate if the instrument is set correctly.  9600 is 2182 default
'''
    This is the approximate transmission rate in bits per second.  Typically it takes about 10 bits to transmit
    a single character (1 start bit + 8 bits for the character + 1 or 2 stop bits), so figure that 
    9600 baud will do about 960 characters per second.

'''
TIME_OUT = 0.50  # don't hang around forever looking for data, but give 2182 time.  0.5 s is good
DATA_LENGTH = 99  #could put this at 9999 and leave it
TEMP_OFFSET = 0.0  # a hack until we calibrate or make direct connection to puck
#check to see if port 0 (COM1) is already open.  If so, you need to close it and reopen:
from serial import SerialException
try:
    ser = serial.Serial("COM1",BAUD,timeout=TIME_OUT)   
except SerialException:   
    print ("Port already open....closing...opening")
    ser.close()
    ser = serial.Serial("COM1",BAUD,timeout=TIME_OUT)

print(ser.name)  #print the name of the com port (should be COM1)
sleep(PAUSE)
print("Flushing Keithley buffers")
ser.flushInput() #flush input buffer, discarding all its contents
sleep(PAUSE)
ser.flushOutput()#flush output buffer, aborting current output 
sleep(PAUSE)

# The following is from Model 2182 p. 2-20
# In order to indicate the end of a line, you'd normally type <ENTER>, which on a (Win)PC would output
# two characters, carriage return (CR)+ linefeed (LF).  In Linux it would be just LF.  The default message
# terminator for the 2182 is simply CR, which is encoded as \r at the end of each message, so that's what
# we use here.
#clear the Keithley 2182 nanovoltmeter
counter = 0
err = writekeithley("*cls",ERR_MSG_ON)
err = writekeithley(":syst:cle",ERR_MSG_ON)
sleep(1)
#err = writekeithley("*cls")
err = writekeithley(":syst:beep:stat off",ERR_MSG_ON) #turn off beep
counter+=1
err = writekeithley(":syst:azer:stat off",ERR_MSG_ON)  #turn off autozero
counter+=1

# Thermocouple
err = writekeithley(":sens:func 'TEMP'",ERR_MSG_ON) # set up Ch2 as thermocouple
err = writekeithley(":unit:temp K",ERR_MSG_ON)
err = writekeithley(":sens:chan 2",ERR_MSG_ON)
err = writekeithley(":sens:temp:tran tc",ERR_MSG_ON)
err = writekeithley(":sens:temp:TC T",ERR_MSG_ON)
#err = writekeithley(":sens:temp:rjun:rsel ext",ERR_MSG_ON) #see p. 2-19
err = writekeithley(":sens:temp:rjun:rsel sim",ERR_MSG_ON) #see p. 2-19
err = writekeithley(":sens:temp:rjun:sim 293.3",ERR_MSG_ON) #see p. 2-19  ****This line needs to be changed if ambient temp differs****
err = writekeithley(":sens:temp:rjun:sim?",ERR_MSG_ON) #see p. 2-19  Check to see if the simulated junction is correct
data = ser.read(DATA_LENGTH)
print("Simulated temperature",data)
err = writekeithley(":sens:temp:TC T",ERR_MSG_ON)
err = writekeithley(":sens:temp:nplc 5",ERR_MSG_ON)# use some signal averaging (see p. 3-7 2182 manual)
#"nplc" means number of averages of the powerline frequency.  This is a clever approach to average each reading
#over an integer number of 60 Hz cycles, reducing noise correlated with the dominant noise source.
err = writekeithley(":sens:temp:rtem?",0) #just for kicks: read internal temperature
data=ser.read(DATA_LENGTH)
print ("The internal temperature of the 2182 thermal junction is: ",float(data))
print("Thermocouple is set up.")
sleep(1)

# Delta voltage readings
#Ohm meter
err = writekeithley(":sens:chan 1",ERR_MSG_ON) # set up Ch1 as voltage (eventually triggered differential voltage)
err = writekeithley(":trig:del 0",ERR_MSG_ON)
err = writekeithley(":sens:func 'VOLT'",ERR_MSG_ON)
err = writekeithley(":sens:volt:delta on",ERR_MSG_ON) # set up Ch1 as differential voltage
err = writekeithley(":sens:volt:nplc 5",ERR_MSG_ON)# use some signal averaging (see p. 3-7)
err = writekeithley(":trig:sour ext",ERR_MSG_ON) # external trigger
sleep(0.5)

err = writekeithley(":syst:cle",ERR_MSG_ON)
err = writekeithley(":sens:data:fres?",ERR_MSG_ON) #we want fresh data, not stale, so read anew
data=ser.read(DATA_LENGTH)
print("Data = ",data)
print ("First read")  #just to see where you are, read channel 1
err = writekeithley(":sens:data:fres?",ERR_MSG_ON) #we want fresh data, not stale, so read anew
data=ser.read(DATA_LENGTH)   
sleep(PAUSE)
if len(data)> 2:
    print(float(data))
#switch channels
#err = writekeithley(":syst:cle",ERR_MSG_ON)
writekeithley(":sens:chan 2",ERR_MSG_ON)
err = writekeithley(":sens:temp:tran tc",ERR_MSG_ON)
writekeithley(":sens:func 'TEMP'",ERR_MSG_ON)

writekeithley(":sens:data:fres?",ERR_MSG_ON)
sleep(0.5) #need a little delay for 2182 to set up
print ("Second read")
data=ser.read(DATA_LENGTH)
sleep(PAUSE)
print("Return bytes from temperature reading: ",data)
if len(data)> 2:
    print(float(data))
#Everything should be working.  Now to loop MAX_COUNT times
#set up the data lists as empty named objects
timestamp =[]
data1 = []
data2 =[]
count=0 #clear the counter to 0, now start the acquisition loop
first_time = True
print("Now scanning over {:d} points".format(MAX_COUNT))
print("Count\tTime\tTemperature\tVoltage\r\n")

#set up both plots
#import matplotlib.pyplot as plt
plt.matplotlib.rcParams.update({'font.size': 22})
fig1 = plt.figure(1, figsize = (8,12))
f_manager = plt.get_current_fig_manager()
f_manager.window.move(-1920,0)  #assumes you have a second screen on the left that is in landscape

plt.title("Voltage vs. Temperature")
plt.xlabel("Temperature(K)")
plt.ylabel("Voltage (V)")

fig2 = plt.figure(2, figsize = (8,12))
plt.xlabel("Data Count")
plt.ylabel("Temperature(K)")
f_manager = plt.get_current_fig_manager()
f_manager.window.move(-1920/2,0)
plt.figure(1)
plt.axis([0,350,0.00,2.0e-4])  #if this isn't big enough, you'll get a blank plot.  The last number needs to be bigger
#than the number that represents the voltage printed while scanning.  It's unlikely that the temperature exceeds 325 K.
plt.figure(2)
plt.title("Temperature vs. Data Count (i.e. time)")
plt.axis([0,MAX_COUNT,0,350])
#bigger than the absolute value of the voltage measured.  You can always zoom the plot with the magnifier glass in order
#to see more detail.


while count<MAX_COUNT: 
    
    err = writekeithley(":sens:chan 1",ERR_MSG_ON)
    err = writekeithley(":sens:func 'VOLT'",ERR_MSG_ON) 
    err = writekeithley(":sens:volt:delta on",ERR_MSG_ON) # set up Ch1 as differential voltage
    #taken from p 5-17 in 2182 manual
    err = writekeithley(":trig:sour ext",ERR_MSG_ON) # external trigger
    sleep(PAUSE)
    err=writekeithley(":sens:data:fres?",ERR_MSG_ON) # read Ch 1 
    sleep(PAUSE)
    data_1=ser.read(DATA_LENGTH)
    if len(data_1)> 2:
        #print(strftime("%a, %d %b %Y %H:%M:%S", localtime())) 
        if (first_time):
            start_time = time.time()  #grab the time at the beginning of the sweep
            first_time = False
        else:
            plt.figure(1)
            plt.scatter(tempK_old,voltage_old, color='b')  #plot old data in blue
            plt.pause(0.001)
            plt.figure(2)
            plt.scatter(count-1,tempK_old, color='b')
        time_now = time.time()-start_time   
        timestamp.append(time_now)
        #print("Raw local time: "+strftime("%H:%M:%S",localtime()))
        voltage = abs(float(data_1))
        data1.append(voltage)
    err = writekeithley(":sens:chan 2",ERR_MSG_ON)
    err = writekeithley(":sens:func 'TEMP'",ERR_MSG_ON) 
    
    #err = writekeithley(":sens:temp:tran tc",ERR_MSG_ON)
    #err = writekeithley(":sens:temp:TC T",ERR_MSG_ON)
    #err = writekeithley(":sens:temp:rjun:rsel sim",ERR_MSG_ON) #see p. 2-19
    #err = writekeithley(":sens:temp:rjun:sim 294.0",ERR_MSG_ON) #see p. 2-19  ****This line needs to be changed if ambient temp differs****
    #err = writekeithley(":trig:sour ext",ERR_MSG_ON) # external trigger
   
    err = writekeithley(":sens:data:fres?",ERR_MSG_ON)  #read Ch 2
    #print("Keithley Channel 2 response on temperature read: ",err)
    
    data_2=ser.read(DATA_LENGTH)
    sleep(PAUSE)
    #print("Raw data 2 reading: ",data_2)
    if len(data_2)> 2:
        print ("   {:d}\t{:5.2f}\t{:6.2f}\t\t{:7.3e}".format(count, time_now, float(data_2),voltage))
     # print("temperature: ".encode(),strftime("%a, %d %b %Y %H:%M:%S", localtime()),count,data.encode())
        data2.append(float(data_2)-TEMP_OFFSET)
        tempK = data_2
        plt.figure(1)
        plt.scatter(tempK,voltage, color='r')  #plot new data in red
        plt.pause(0.001)
        plt.figure(2)
        plt.scatter(count,tempK, color = 'r')
        tempK_old = tempK
        voltage_old = voltage
        
        plt.pause(0.001)
        count+=1  #increment the counter after presumably getting both reads

ser.close() #close the COM port when finished
plt.show()
print("Finished taking data and closing serial port")
#
#Now that we're done taking data, we can process the two arrays data1 and data2
# Write the data out as a two-column T, V text file
import numpy as np  #we need the savetxt package

#In case you interrupt data taking but want to write out the results, you'll find that one of your arrays
#is longer than the other.  This would generate an error, because savetxt wants the two arrays to have identical 
#lengths.  Here's a cheap way to get around the problem without truncating the data sets:
today_str = strftime("%d%m%y_%H.%M",localtime())  #grab the local time
fname = "Superconductivity "+today_str+".dat"
l1=len(data1)
l2 =len(data2)
if (l2 < l1):  
    data2.append(296.3-TEMP_OFFSET)#add a dummy final value on the end of data2
elif (l1 < l2):
    data1.append(0.0) #add a dummy final value on the end of data1
tempK_arr = np.array(data2,float)
voltage_arr =abs(np.array(data1,float))
np.savetxt(fname,np.column_stack((timestamp,tempK_arr,voltage_arr)),delimiter="\t",fmt= "%9.4e", header=headertext)
print("Finished writing file as "+fname)
winsound.Beep(2093, 800)
winsound.Beep(1760,1500)

Using matplotlib backend: Qt5Agg
COM1
Flushing Keithley buffers
Simulated temperature b'2.93E+02\r'
The internal temperature of the 2182 thermal junction is:  20.15
Thermocouple is set up.
Data =  b'-9.75112823E-05\r'
First read
-9.75093811e-05
Second read
Return bytes from temperature reading:  b'+2.30744924E+02\r'
230.744924
Now scanning over 600 points
Count	Time	Temperature	Voltage

   0	 0.00	230.80		9.751e-05


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


   1	 1.39	230.84		9.746e-05
   2	 2.68	230.87		9.747e-05
   3	 3.97	230.91		9.759e-05
   4	 5.25	230.94		9.759e-05
   5	 6.54	230.97		9.751e-05
   6	 7.83	231.00		9.762e-05
   7	 9.13	231.04		9.753e-05
   8	10.42	231.08		9.757e-05
   9	11.73	231.12		9.760e-05
   10	13.03	231.16		9.752e-05
   11	14.34	231.20		9.756e-05
   12	15.64	231.24		9.805e-05
   13	16.96	231.28		9.773e-05
   14	18.27	231.33		9.762e-05
   15	19.58	231.41		9.788e-05
   16	20.89	231.56		9.768e-05
   17	22.19	231.77		9.772e-05
   18	23.49	231.98		9.771e-05
   19	24.79	232.21		9.761e-05
   20	26.11	232.47		9.758e-05
   21	27.42	232.73		9.777e-05
   22	28.74	233.06		9.790e-05
   23	30.05	233.33		9.827e-05
   24	31.35	232.94		9.726e-05
   25	32.66	231.21		9.855e-05
   26	33.99	228.88		9.625e-05
   27	35.29	226.56		9.890e-05
   28	36.59	224.40		9.594e-05
   29	37.90	222.41		9.818e-05
   30	39.24	220.65		9.593e-05
   31	40.57	219.07		9.763e-05
   32	41.91	217.66		9.514e-05
   33	43.23	216.38		9.655e-05
   34	44.57	215.23	

### If you quit the above cell early (by hitting the stop button), you'll want to run this cell to save data

In [ ]:
#Now that we're done taking data, we can process the two arrays data1 and data2
# Write the data out as a two-column T, V text file
%matplotlib
import matplotlib.pyplot as plt
import serial  #pySerial folder needs to be put in the directory pythonXX\lib\site-packages\serial
ser.close() #close the COM port when finished

import numpy as np  #we need the savetxt package
#from time import sleep, localtime,strftime #some timekeeping functions
today_str = strftime("%d%m%y_%H.%M",localtime())  #grab the local time
fname = "Superconductivity "+today_str+".dat"
headertext = "Time, Temperature, Voltage for 10 mA drive current"

#In case you interrupt data taking but want to write out the results, you'll find that one of your arrays
#is longer than the other.  This would generate an error, because savetxt wants the two arrays to have identical 
#lengths.  Here's a cheap way to get around the problem without truncating the data sets:
l1=len(data1)
l2 =len(data2)
if (l2 < l1):  
    data2.append(296.3-TEMP_OFFSET)#add a dummy final value on the end of data2
elif (l1 < l2):
    data1.append(0.0) #add a dummy final value on the end of data1
np.savetxt(fname,np.column_stack((timestamp,data2,data1)),delimiter="\t",fmt="%13.9e",header=headertext)
plt.close('all')

### Now to plot data in Python, assuming you've got the arrays data1 and data2 from above,  you simply invoke the plot routine:

Or if you'd like to export the data, you'll find it under Superconductivity +<timestamp>.dat. which should be readable in Excel.  But why not learn some simple Python plotting routines now?

We've stored the data as a list, which could have all sorts of objects and data types in it.  So let's first convert it to a floating point array.  Then we can manipulate it, say by dividing by the current, so as to get resistance.


In [5]:
temp_data = np.array(data2,float)
voltage_data = np.array(data1,float)

resist_data = voltage_data/10e-3  #now it's ohms, assuming 10 mA current

NameError: name 'data2' is not defined

### To annotate your plot, you'll want a title.  Try this block:

In [ ]:
fig1 = plt.figure(1, figsize = (12,12))
f_manager = plt.get_current_fig_manager()
f_manager.window.move(-1920,0)  #assumes you have a second screen on the left that is in landscape

plt.matplotlib.rcParams.update({'font.size': 12})  #smaller font size
plt.title("YBCO resistance vs. absolute temperature 10 mA excitation\n")
plt.xlabel("Degrees K")
plt.ylabel("Resistance in Ohms")
plt.plot(temp_data,resist_data, '*k', ms = 12) #big black stars, ms = 'marker size'
plt.show()

### If you'd rather plot it with green pluses, then you'd do something like this:

In [ ]:
fig2 = plt.figure(2, figsize = (12,12))
f_manager = plt.get_current_fig_manager()
f_manager.window.move(-1920/2,0)  #assumes you have a second screen on the left that is in landscape
plt.title("YBCO resistance vs. absolute temperature 10 mA excitation\n")
plt.xlabel("Degrees K")
plt.ylabel("Resistance in Ohms")
plt.plot(temp_data,resist_data, '+g', ms = 6) #green plus signs

plt.show()

###  If you want to plot only the rising temperature data, you can select all the points after let's say the 400th point
You do this by restricting the range of temp_data and resist_data above:

plot(temp_data[400:],resist_data[400:], '+g') #green plus signs

### Note that the plot now is from points with index 400 until the end.  If you wanted only 400 to 799, you'd say
plot (temp_data[400:800], resist_data[400:800])

###  And it's always a good idea to close the serial port connection when you are done:
ser.close() #close the COM port when finished

In [ ]:
ser.close() #close the COM port when finished